In [9]:
%pip install opencv-python
%pip install imageio
%pip install imutils
%pip install IPython



Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#added'


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#added'


Installations added by Afia

In [27]:
%pip install pandas 
%pip install utils 
%pip install matplotlib
%pip install seaborn
%pip install feature_engine
%pip install keras
%pip install tensorflow
%pip install hyperopt
%pip install mlflow

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached mlflow-2.12.1-py3-none-any.whl.metadata (29 kB)
  Using cached alembic-1.13.1-py3-none-any.whl.metadata (7.4 kB)
  Using cached docker-7.0.0-py3-none-any.whl.metadata (3.5 kB)
  Using cached graphene-3.3-py2.py3-none-any.whl.metadata (7.7 kB)
  Using cached querystring_parser-1.2.4-py2.py3-none-any.whl.metadata (559 bytes)
  Using cached sqlparse-0.5.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached waitress-3.0.0-py3-none-any.whl.metadata (4.2 kB)
  Using cached Mako-1.3.3-py3-none-any.whl.metadata (2.9 kB)
  Usin

In [31]:
# Import Packages for project

# Standard Libraries
import csv
import cv2
import imageio
from IPython.display import YouTubeVideo
import glob
import numpy as np
import os
import pandas as pd
import re
import utils

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Algorithms, Modeling and Data Pre-processing
import feature_engine
from feature_engine.encoding import OrdinalEncoder
from feature_engine.transformation import YeoJohnsonTransformer
from scipy.stats import anderson, chi2_contingency
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score,precision_score, roc_auc_score,recall_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Deep Learning
import keras
from keras import layers
from keras.layers import RandomFlip, RandomRotation, Rescaling, BatchNormalization, Conv2D, MaxPooling2D, Dense, Input
from keras.models import Model, Sequential
from keras.optimizers import Adam, SGD
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Model Optimization and Hyperparameter Tuning
import hyperopt
from hyperopt import STATUS_OK, Trials, fmin, tpe, hp
import mlflow

import tensorboard

In [32]:
# Obtain the Data
filepath = '../data/K400/video_annotations.csv'
raw_csv = pd.read_csv(filepath)
k400_df = pd.DataFrame(raw_csv)

k400_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19906 entries, 0 to 19905
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   label       19906 non-null  object
 1   youtube_id  19906 non-null  object
 2   time_start  19906 non-null  int64 
 3   time_end    19906 non-null  int64 
 4   split       19906 non-null  object
 5   is_cc       19906 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 4.2 MB


In [33]:
# initial inspection of complete dataframe
k400_df.head

<bound method NDFrame.head of            label   youtube_id  time_start  time_end split  is_cc
0      abseiling  0wR5jVB-WPk         417       427   val      0
1      abseiling  3caPS4FHFF8          36        46   val      0
2      abseiling  3yaoNwz99xM          62        72   val      1
3      abseiling  6IbvOJxXnOo          47        57   val      0
4      abseiling  6_4kjPiQr7w         191       201   val      0
...          ...          ...         ...       ...   ...    ...
19901      zumba  w5hbJLVhZDI          93       103   val      0
19902      zumba  xDd6uIBeMEA           1        11   val      0
19903      zumba  XWvGn7eI04A          12        22   val      0
19904      zumba  yGdQwxP5koA          83        93   val      0
19905      zumba  ZVDR2od1gn8          37        47   val      0

[19906 rows x 6 columns]>

In [34]:
# Check for null values/percentage of null values:

k400_df.isna().mean()

label         0.0
youtube_id    0.0
time_start    0.0
time_end      0.0
split         0.0
is_cc         0.0
dtype: float64

In [35]:
k400_df.isna().sum()

label         0
youtube_id    0
time_start    0
time_end      0
split         0
is_cc         0
dtype: int64

### Observations 

1. No null values
2. Over 240k Observations
3. 6 Attributes of string/int datatypes

Afia's notes from earlier discussion with Armando on Tuesday (April 23) is that we should ignore split and is_cc attributes/columns

In [36]:
# check for dup
num_unique = k400_df.nunique()
num_unique

label           400
youtube_id    19906
time_start      842
time_end        842
split             1
is_cc             2
dtype: int64

### Observations

1. THere are 400 unique labels
2. There are about 20K youtube_id with only about 850 videos
3. Videos duration is only 10 seconds as annotated by the difference between time_start and time_end values

**Next Steps**: to reduce the dimensionality, I need to create a function that will map a video file to a youtube id value in the video_annotations.csv file and create a new dataframe where we have a match. Data Cleaning required on the names of the video files

In [37]:
youtube_id_values = k400_df['youtube_id']
print(f"Total Youtube ID Values in Dataset: {youtube_id_values.count()}")

Total Youtube ID Values in Dataset: 19906


In [38]:
# Get the amount of unique youtube_id
number_unique_id = youtube_id_values.nunique()
print(f"Unique Youtube ID Values: {number_unique_id}")

Unique Youtube ID Values: 19906


In [39]:
# Check for unique values
unique_youtube_id = youtube_id_values.unique()
unique_youtube_id

array(['0wR5jVB-WPk', '3caPS4FHFF8', '3yaoNwz99xM', ..., 'XWvGn7eI04A',
       'yGdQwxP5koA', 'ZVDR2od1gn8'], dtype=object)

Commenting out Tim's code to implement Pseudo Code and/or associated code with ChatGPT results

In [ ]:
# # Get the number of video files we are working with

# def count_video_files(directory):
#     """
#     Purpose - to get a video file count within a given directory
#     Arguments - directory variable that holds the filepath to a video directory
#     Returns - video_count of type integer
    
#     """
#     # Set the allowed video file extensions
#     video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.wmv', '.flv']

#     # Initialize the count
#     video_count = 0

#     # Iterate through all files in the directory
#     for file_path in glob.glob(os.path.join(directory, '*')):
#         # Check if the file has a video file extension
#         if os.path.isfile(file_path) and any(file_path.lower().endswith(ext) for ext in video_extensions):
#             video_count += 1

#     return video_count


In [ ]:
# # Test Functionality and return video count

# # Provide the directory path to count video files
# directory_path = '../data/K400/videos'

# # Call the function to count video files
# num_videos = count_video_files(directory_path)
# print(f'Total number of video files: {num_videos} videos present')

### Video Observations

1. There seems to be a match with the youtube_id values in the video_annotations.csv file and the initial naming convention of the video files.
2. The videofile names have a timestamp that highlights how the 10second video frame was captured. 

**Next Steps**: In order to load in local video data correctly, I need to perform regular expressions to rename the video files to exclude the timestamps.

In [ ]:
# def remove_timestamp(filename):
#     """
#     Purpose: to remove the timestampe suffix at the end of our local video files
#     Arguments: filename 
#     Retunrs: Cleaned filename
#     """
#     # Split the filename by underscores
#     parts = filename.rsplit('_')

#     # Filter out parts that are likely numbers
#     cleaned_parts = [part for part in parts if not part.isdigit()]

#     # Join the cleaned parts with underscores to form the new filename
#     cleaned_filename = '_'.join(cleaned_parts)

#     return cleaned_filename # Remove leading/trailing whitespaces


In [ ]:
# def rename_files(directory):
#     """
#     Purpose: To rename all the local video files in our directory for future loading 
#     Arguments: Filepath to the video directory
#     Returns: None

#     Other Functions: Calls the remove_timestamp()
#     """
#     # Iterate through all files in the directory
#     for filename in os.listdir(directory):
#         # Check if the file is a regular file (not a directory)
#         if os.path.isfile(os.path.join(directory, filename)):
#             # Remove timestamp from the filename
#             new_filename = remove_timestamp(filename)
#             # Rename the file if the filename has changed
#             if new_filename != filename:
#                 os.rename(os.path.join(directory, filename),
#                           os.path.join(directory, new_filename))

In [ ]:
# # Test
# video_directory = "../data/K400/videos"

# rename_files(video_directory)

In [ ]:
# video_directory = '../data/K400/videos'

# # Iterate through each YouTube ID
# for youtube_id in youtube_id_values:
#     # Find the corresponding video file in the directory
#     for filename in os.listdir(video_directory):
#         if youtube_id in filename:
#             # Extract the file extension
#             file_extension = os.path.splitext(filename)[1]

#             # Construct the new file name without the timestamp
#             new_filename = youtube_id + file_extension

#             # Construct the full paths for old and new files
#             old_filepath = os.path.join(video_directory, filename)
#             new_filepath = os.path.join(video_directory, new_filename)

#             # Rename the file
#             os.rename(old_filepath, new_filepath)
#             print(f'Renamed {filename} to {new_filename}')
#             break

### Observations

Removed the start_time portion of the timestamp, but left the end_time timestamp in the video file name

*** Ending of section COMMENTED OUT by Afia on Tuesday (April 23) ****

1st Attempt at running this block of code below had an issue with the video_name since there is no column in video_annotations.csv

In [41]:
# import os
# import pandas as pd

# # Function to compare contents of CSV file with contents of folder
# def compare_csv_with_folder(csv_file, folder_path):
#     # Read CSV file
#     df = pd.read_csv(csv_file)
    
#     # Loop through each row in the dataframe
#     for index, row in df.iterrows():
#         youtube_id = row['youtube_id']
#         video_name = row['video_name']
        
#         # Search for video with youtube_id in folder
#         for filename in os.listdir(folder_path):
#             if youtube_id in filename:
#                 # Rename video file
#                 old_file_path = os.path.join(folder_path, filename)
#                 new_file_path = os.path.join(folder_path, f"{youtube_id}.mp4")
#                 os.rename(old_file_path, new_file_path)
#                 print(f"Renamed {filename} to {youtube_id}.mp4")
#                 # Update dataframe
#                 df.at[index, 'video_name'] = f"{youtube_id}.mp4"
#                 break
    
#     # Write updated dataframe to CSV
#     df.to_csv(csv_file, index=False)
#     print("CSV file updated successfully")

# # Specify paths to CSV file and folder
# csv_file = "../data/K400/video_annotations.csv"
# folder_path = "../data/K400/videos"

# # Call function to compare CSV with folder
# compare_csv_with_folder(csv_file, folder_path)


Intermediary Step to get the video filenames into a csv

In [42]:
import os
import csv

# Function to get video names from a folder and save them in a CSV file
def videos_to_csv(folder_path, csv_file):
    # Get list of video files in the folder
    video_files = [filename for filename in os.listdir(folder_path) if filename.endswith(('.mp4', '.avi', '.mov', '.mkv'))]

    # Write video names to CSV file
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['video_name'])
        for video_name in video_files:
            writer.writerow([video_name])
    print("CSV file created successfully")

# Specify path to the folder containing videos
folder_path = "../data/K400/videos"

# Specify path to the CSV file to be created
csv_file = "../data/K400/video_names.csv" 

# Call the function to extract video names and save them in the CSV file
videos_to_csv(folder_path, csv_file)


CSV file created successfully


Observations: Only 1,138 rows of video_names were extracted from the folder

2nd Attempt of previous code block being run here. Found out at 7:56 PM ET on Tuesday (April 23) that Tim has already moved on from this issue

In [ ]:
# import os
# import pandas as pd

# # Function to compare contents of CSV file with contents of folder
# def compare_csv_with_folder(csv_file, folder_path):
#     # Read CSV file
#     df = pd.read_csv(csv_file)
    
#     # Loop through each row in the dataframe
#     for index, row in df.iterrows():
#         youtube_id = row['youtube_id']
#         video_name = row['video_name']
        
#         # Search for video with youtube_id in folder
#         for filename in os.listdir(folder_path):
#             if youtube_id in filename:
#                 # Rename video file
#                 old_file_path = os.path.join(folder_path, filename)
#                 new_file_path = os.path.join(folder_path, f"{youtube_id}.mp4")
#                 os.rename(old_file_path, new_file_path)
#                 print(f"Renamed {filename} to {youtube_id}.mp4")
#                 # Update dataframe
#                 df.at[index, 'video_name'] = f"{youtube_id}.mp4"
#                 break
    
#     # Write updated dataframe to CSV
#     df.to_csv(csv_file, index=False)
#     print("CSV file updated successfully")

# # Specify paths to CSV file and folder
# csv_file = "../data/K400/video_annotations.csv"
# folder_path = "../data/K400/videos"

# # Call function to compare CSV with folder
# compare_csv_with_folder(csv_file, folder_path)


In [46]:
import os
import pandas as pd

# Function to rename video files based on youtube_id
def rename_videos(video_annotations_file, video_names_file, folder_path):
    # Read video annotations CSV
    video_annotations_df = pd.read_csv(video_annotations_file)

    # Read video names CSV
    video_names_df = pd.read_csv(video_names_file)

    # Iterate through each row in video annotations
    for index_ann, row_ann in video_annotations_df.iterrows():
        youtube_id = row_ann['youtube_id']
        
        # Iterate through each row in video names
        for index_name, row_name in video_names_df.iterrows():
            video_name = row_name['video_name']
            
            # Check if youtube_id is in video_name
            if str(youtube_id) in video_name:
                # Rename video file
                old_file_path = os.path.join(folder_path, video_name)
                new_file_path = os.path.join(folder_path, f"{youtube_id}.mp4")
                os.rename(old_file_path, new_file_path)
                print(f"Renamed {video_name} to {youtube_id}.mp4")
                break

# Specify paths to CSV files and folder
video_annotations_file = r"../data/K400/video_annotations.csv"
video_names_file = r"../data/K400/video_names.csv"
folder_path = r"../data/K400/videos"
# Call function to rename video files
rename_videos(video_annotations_file, video_names_file, folder_path)


Renamed 0wR5jVB-WPk_000417_000427.mp4 to 0wR5jVB-WPk.mp4


OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: '../data/K400/videos\\#NAME?' -> '../data/K400/videos\\#NAME?.mp4'

### Define Hyperparameters

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 10

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

### Data Preparation

In [ ]:
split_percent = .30

# Split the k400_df into test and train df

# Split the dataframe into train and test using pd.sample()
test_df = k400_df.sample(frac=split_percent, random_state=42)
train_df = k400_df.drop(test_df.index)

# Reset the index of the new dataframes
test_df.reset_index(drop=True, inplace=True)
train_df.reset_index(drop=True, inplace=True)

In [ ]:
test_df

In [ ]:
train_df

### Notes

train_df has 13934 Rows with 6 Attributes

test_df has 5972 rows

In [ ]:

def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]

In [ ]:
def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)